In [8]:
def Simulate_Heston(n_paths, method):
    # Change in BM
    dW1=npr.normal(0,np.sqrt(dt),(n_paths,steps))
    dW2=rho*dW1+np.sqrt(1-rho**2)*npr.normal(0,np.sqrt(dt),(n_paths,steps))

    # Initialize S1, S2, V, integral of V and L
    S1 = np.empty([n_paths,steps+1])
    V = np.empty([n_paths,steps+1])
    S1[:,0] = S0
    V[:,0] = V0
    
    Int_V = np.empty([n_paths,steps+1])
    S2 = np.empty([n_paths,steps+1])

    Int_V[:,0] = 0
    S2[:,0] = L(0,V0)
    
    # Simulates Heston paths
    
    # Moment Matching
    if (method == "moment matching"):
        for i in range(steps):
            X = np.log(S1[:,i])
            X = X+(mu-0.5*V[:,i])*dt+np.sqrt(V[:,i])*dW1[:,i]
            S1[:,i+1] = np.exp(X)
            Gamma2 =  1/dt*np.log(1+(0.5*sigma**2/kappa*V[:,i]*(1-np.exp(-2*kappa*dt)))/((np.exp(-kappa*dt)*V[:,i]+(1-np.exp(-kappa*dt))*theta)**2))
            V[:,i+1] = (np.exp(-kappa*dt)*V[:,i]+(1-np.exp(-kappa*dt))*theta)*np.exp(-0.5*Gamma2*dt+np.sqrt(Gamma2)*dW2[:,i])
            Int_V[:,i+1] = Int_V[:,i]+dt*(V[:,i]+V[:,i+1])/2
            S2[:,i+1] = Int_V[:,i+1] + L(dt*(i+1),V[:,i+1])
            
    # Full truncation
    if (method == "full truncation"):
        Vt = np.empty([n_paths,steps+1]) # V_tilde
        Vt[:,0] = V0
        for i in range(steps):
            X = np.log(S1[:,i])
            X = X+(mu-0.5*V[:,i])*dt+np.sqrt(V[:,i])*dW1[:,i]
            S1[:,i+1] = np.exp(X)
            Vt[:,i+1] = Vt[:,i]+kappa*(theta-np.maximum(Vt[:,i],0))*dt+sigma*np.sqrt(np.maximum(Vt[:,i],0))*dW2[:,i]
            V[:,i+1] = np.maximum(Vt[:,i+1],0)
            Int_V[:,i+1] = Int_V[:,i]+dt*(V[:,i]+V[:,i+1])/2
            S2[:,i+1] = Int_V[:,i+1] + L(dt*(i+1),V[:,i+1])
            
    return S1, S2, V

In [2]:
def MCprice_BS(option_type, n_paths):
    xi = npr.normal(0,np.sqrt(dt),(n_paths,steps)) # Makes an (n x steps) matrix full of N(0,1/steps) realizations
    W = np.apply_along_axis(np.cumsum,1,xi) # Cumulatively adds the realizations along each row
    W = np.concatenate((np.zeros((n_paths,1)),W),1) # Adds zeros in the first column of the matrix
    drift = np.linspace(0,(r-0.5*sigma**2)*T,steps+1) # Makes a (steps+1) length vector of equidistant values, ranging from 0 to mu
    drift = np.reshape(drift,(1,steps+1)) # Converts the array into a matrix (with only 1 row)
    drift = np.repeat(drift,n_paths,axis=0) # Repeats the row n times
    S = S0 * np.exp(drift + sigma * W) # Calculates S(t) at all timesteps and paths
    
    if (option_type == 'call'):
        payoff = np.mean(np.maximum(S[:,-1]-K,0))
    if (option_type == 'digital'):
        payoff = np.mean(S[:,-1]>K)
    if (option_type == 'asian'):
        payoff = np.mean(np.maximum(np.mean(S[:,1:],axis=-1)-K,0))
    if (option_type == 'lookback'):
        payoff = np.mean(S[:,-1]-np.min(S,axis=-1))
    return payoff*np.exp(-r*T)
    

In [ ]:
def avg_asian(n_paths):
    xi = npr.normal(0,np.sqrt(dt),(n_paths,steps)) # Makes an (n x steps) matrix full of N(0,1/steps) realizations
    W = np.apply_along_axis(np.cumsum,1,xi) # Cumulatively adds the realizations along each row
    W = np.concatenate((np.zeros((n_paths,1)),W),1) # Adds zeros in the first column of the matrix
    drift = np.linspace(0,(r-0.5*sigma**2)*T,steps+1) # Makes a (steps+1) length vector of equidistant values, ranging from 0 to mu
    drift = np.reshape(drift,(1,steps+1)) # Converts the array into a matrix (with only 1 row)
    drift = np.repeat(drift,n_paths,axis=0) # Repeats the row n times
    S = S0 * np.exp(drift + sigma * W) # Calculates S(t) at all timesteps and paths
    
    avg = np.empty([steps])
    Sbar = np.mean(S[:,1:],axis=-1)
    indicator = (Sbar > K)
    for i in range(steps):
        avg[i] = np.sum(indicator*np.sum(S[:,(i+1):],axis=-1)/steps)/n_paths
    return avg

In [ ]:
def MCprice_Heston(option_type, n_paths, method):
    
    # Change in BM
    dW1=npr.normal(0,np.sqrt(dt),(n_paths,steps))
    dW2=rho*dW1+np.sqrt(1-rho**2)*npr.normal(0,np.sqrt(dt),(n_paths,steps))

    # Initialize S1, S2, V, integral of V and L
    S1 = np.empty([n_paths,steps+1])
    V = np.empty([n_paths,steps+1])
    S1[:,0] = S0
    V[:,0] = V0
    
    #Int_V = np.empty([n_paths,steps+1])
    #S2 = np.empty([n_paths,steps+1])
    #Int_V[:,0] = 0
    #S2[:,0] = L(0,V0)
    
    # Simulates Heston paths
    
    # Moment Matching
    if (method == "moment matching"):
        for i in range(steps):
            X = np.log(S1[:,i])
            X = X+(r-0.5*V[:,i])*dt+np.sqrt(V[:,i])*dW1[:,i]
            S1[:,i+1] = np.exp(X)
            Gamma2 =  1/dt*np.log(1+(0.5*sigma**2/kappa*V[:,i]*(1-np.exp(-2*kappa*dt)))/((np.exp(-kappa*dt)*V[:,i]+(1-np.exp(-kappa*dt))*theta)**2))
            V[:,i+1] = (np.exp(-kappa*dt)*V[:,i]+(1-np.exp(-kappa*dt))*theta)*np.exp(-0.5*Gamma2*dt+np.sqrt(Gammma2)*dW2[:,i])
            #Int_V[:,i+1] = Int_V[:,i]+dt*(V[:,i]+V[:,i+1])/2 # By the trapzoid rule
            #S2[:,i+1] = Int_V[:,i+1] + L(dt*(i+1),V[:,i+1])
            
    # Full truncation
    if (method == "full truncation"):
        for i in range(steps):
            X = np.log(S1[:,i])
            X = X+(r-0.5*np.maximum(V[:,i],0))*dt+np.sqrt(np.maximum(V[:,i],0))*dW1[:,i]
            S1[:,i+1] = np.exp(X)
            V[:,i+1] = V[:,i]+kappa*(theta-np.maximum(V[:,i],0))*dt+sigma*np.sqrt(np.maximum(V[:,i],0))*dW2[:,i]
            #Int_V[:,i+1] = Int_V[:,i]+dt*(V[:,i]+V[:,i+1])/2
            #S2[:,i+1] = Int_V[:,i+1] + L(dt*(i+1),V[:,i+1])
    
    if (option_type == 'call'):
        payoff = np.mean(np.maximum(S1[:,-1]-K,0))
    if (option_type == 'digital'):
        payoff = np.mean(S1[:,-1]>K)
    if (option_type == 'asian'):
        payoff = np.mean(np.maximum(np.mean(S1[:,1:],axis=-1)-K,0))
    if (option_type == 'lookback'):
        payoff = np.mean(S1[:,-1]-np.min(S1,axis=-1))
    return payoff*np.exp(-r*T)
    